<a href="https://colab.research.google.com/github/colourdelete/stuff/blob/main/Copy_of_preprocessing_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (Data Preprocessor) Predict Wildfires From Stuff Like Temp, Humidity, etc

## Import Stuff


In [50]:
#@title Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import requests
import datetime
import time
import os

In [51]:
#@title API Setting

owm_api_key = '3bfaec2c38caf5943689747c2ddd3755' #@param{type:'string'}

In [52]:
#@title Get & Process Data
def get_process_data():
  fire_dataset_url = 'https://cwfis.cfs.nrcan.gc.ca/downloads/nfdb/fire_pnt/current_version/NFDB_point_txt.zip' #@param{type:'string'}
  fire_path = 'datasets/NFDB_point_20200820.txt' #@param{type:'string'}
  fire_cache_dir = '.' #@param{type:'string'}
  fire_extract = True #@param{type:'boolean'}
  tf.keras.utils.get_file(os.path.splitext(fire_dataset_url)[1], fire_dataset_url,
                          extract=fire_extract, cache_dir=fire_cache_dir)
  ! head datasets/NFDB_point_20200820.txt
  fire_df = pd.read_csv(fire_path)
  weather_dataset_url = 'https://cwfis.cfs.nrcan.gc.ca/downloads/nfdb/fire_pnt/current_version/NFDB_point_txt.zip' #@param{type:'string'}
  weather_path = 'datasets/NFDB_point_20200820.txt' #@param{type:'string'}
  weather_cache_dir = '.' #@param{type:'string'}
  weather_extract = True #@param{type:'boolean'}
  tf.keras.utils.get_file(os.path.splitext(weather_dataset_url)[1], weather_dataset_url,
                          extract=weather_extract, cache_dir=weather_cache_dir)
  weather_df = pd.read_csv(weather_path)
  return fire_df, weather_df


fire_df, weather_df = get_process_data()

FID,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,ATTK_DATE,OUT_DATE,DECADE,SIZE_HA,CAUSE,PROTZONE,FIRE_TYPE,MORE_INFO,CFS_REF_ID,CFS_NOTE1,CFS_NOTE2,ACQ_DATE,ECODISTRIC,ECOREGION,ECOZONE,CFS_ECOZ,SRC_AGY2
0,BC,1953-G00041, ,59.963000000000001,-128.172000000000000,1953,5,26,1953-05-26 00:00:00,,,1950-1959,8.000000000000000,H, ,Fire, ,BC-1953-1953-G00041, , ,2020-05-05 00:00:00,925,181,12,12,BC
1,BC,1950-R00028, ,59.317999999999998,-132.172000000000000,1950,6,22,1950-06-22 00:00:00,,,1950-1959,8.000000000000000,L, ,Fire, ,BC-1950-1950-R00028, , ,2020-05-05 00:00:00,905,177,12,12,BC
2,BC,1950-G00026, ,59.875999999999998,-131.922000000000000,1950,6,4,1950-06-04 00:00:00,,,1950-1959,12949.900000000000000,H, ,Fire, ,BC-1950-1950-G00026, , ,2020-05-05 00:00:00,905,177,12,12,BC
3,BC,1951-R00097, ,59.759999999999998,-132.807999999999990,1951,7,15,1951-07-15 00:00:00,,,1950-1959,241.099999999999990,H, ,Fire, ,BC-1951-1951-R00097, , ,2020-05-05 00:00:00,905,177,12,12,BC


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,11,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [70]:
#@title Merge Data into a Datum

for fire_row in fire_df.iterrows():
  print(len(fire_row[1]))
  # lat        = fire_row['LATITUDE']
  # lon        = fire_row['LONGITUDE']
  # y          = fire_row['YEAR']
  # m          = fire_row['MONTH']
  # d          = fire_row['DAY']
  lat        = fire_row[1][4]
  lon        = fire_row[1][5]
  y          = fire_row[1][6]
  m          = fire_row[1][7]
  d          = fire_row[1][8]
  middle_dt  = datetime.datetime(y, m, d)
  middle     = time.mktime(middle_dt.timetuple())
  data_range = 2 #@param {type:'integer'}
  start      = time.mktime((middle_dt - datetime.timedelta(-data_range)).timetuple())
  end        = time.mktime((middle_dt - datetime.timedelta(data_range)).timetuple())
  # https://openweathermap.org/history#geo
  print(f'http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={owm_api_key}')
  data = requests.get(f'http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={owm_api_key}').json()
  if data['cod'] != 200:
    print('skip, cod non-200.', data)
  else:
    print(data['list'])


27
http://history.openweathermap.org/data/2.5/history/city?lat=59.963&lon=-128.172&type=hour&start=-523756800.0&end=-524102400.0&appid=3bfaec2c38caf5943689747c2ddd3755
skip, cod non-200. {'cod': 401, 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}
27
http://history.openweathermap.org/data/2.5/history/city?lat=59.318&lon=-132.172&type=hour&start=-616118400.0&end=-616464000.0&appid=3bfaec2c38caf5943689747c2ddd3755
skip, cod non-200. {'cod': 401, 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}
27
http://history.openweathermap.org/data/2.5/history/city?lat=59.876&lon=-131.922&type=hour&start=-617673600.0&end=-618019200.0&appid=3bfaec2c38caf5943689747c2ddd3755
skip, cod non-200. {'cod': 401, 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}
27
http://history.openweathermap.org/data/2.5/history/city?lat=59.76&lon=-132.808&type=hour&start=-582595200.0&e

KeyboardInterrupt: ignored